# PixelPrism Math Basics

This notebook walks through the foundational math containers used across PixelPrism:
- `DType` captures scalar/element types.
- `Shape` stores symbolic tensor dimensions.
- `Value` ties user data with `Shape`/`DType` metadata.
Each section below demonstrates typical usage patterns and key behaviors.

In [1]:
from pixelprism.math.dtype import DType
from pixelprism.math.shape import Shape
from pixelprism.math.value import Value

## DType

In [2]:
floats = (DType.FLOAT32, DType.FLOAT64)
ints = (DType.INT32, DType.INT64)
summary = []
for dt in DType:
    summary.append({
        'name': dt.name,
        'value': dt.value,
        'is_float': dt.is_float,
        'is_int': dt.is_int,
        'is_bool': dt.is_bool
    })
summary

[{'name': 'FLOAT64',
  'value': 'float64',
  'is_float': True,
  'is_int': False,
  'is_bool': False},
 {'name': 'FLOAT32',
  'value': 'float32',
  'is_float': True,
  'is_int': False,
  'is_bool': False},
 {'name': 'INT64',
  'value': 'int64',
  'is_float': False,
  'is_int': True,
  'is_bool': False},
 {'name': 'INT32',
  'value': 'int32',
  'is_float': False,
  'is_int': True,
  'is_bool': False},
 {'name': 'BOOL',
  'value': 'bool',
  'is_float': False,
  'is_int': False,
  'is_bool': True}]

The helper method `DType.promote(a, b)` mimics how binary ops pick a resulting dtype. 
The ordering favors more expressive types, so mixing `FLOAT32` with `INT32` yields `FLOAT32`.

In [3]:
DType.promote(DType.FLOAT64, DType.INT32)

<DType.FLOAT64: 'float64'>

## Shape

In [4]:
vector = Shape.vector(3)
matrix = Shape.matrix(2, 4)
unknown = Shape((None, 4))
{'vector': vector, 'matrix': matrix, 'unknown': unknown, 'matrix_rank': matrix.rank, 'unknown_size': unknown.size}

{'vector': Shape((3,)),
 'matrix': Shape((2, 4)),
 'unknown': Shape((None, 4)),
 'matrix_rank': 2,
 'unknown_size': None}

Shapes can verify broadcast/elementwise compatibility and produce derived shapes for ops like `matmul`.

In [5]:
lhs = Shape((2, 3, 4))
rhs = Shape((2, 4, 5))
result = lhs.matmul_result(rhs)
compat = lhs.merge_elementwise(lhs)
{'matmul': result, 'elementwise': compat, 'can_reshape': lhs.can_reshape(Shape((4, 3, 4)))}

{'matmul': Shape((2, 3, 5)),
 'elementwise': Shape((2, 3, 4)),
 'can_reshape': False}

In [9]:
from pixelprism.math import SymbolicDim
N = SymbolicDim('N')
shape = Shape((N, N))
print(shape)
print(shape.rank)

Shape(SymbolicDim(name='N')xSymbolicDim(name='N'))
2


## Value

In [8]:
data = [[1.0, 2.0], [3.0, 4.0]]
value = Value(data=data, shape=Shape.matrix(2, 2), dtype=DType.FLOAT32)
{'shape': value.shape, 'dtype': value.dtype, 'mutable': value.mutable, 'data': value.get()}

{'shape': Shape((2, 2)),
 'dtype': <DType.FLOAT32: 'float32'>,
 'mutable': True,
 'data': [[1.0, 2.0], [3.0, 4.0]]}

The value enforces the declared shape when wrapping nested Python data. Attempting to assign data
with mismatched dimensions raises `ValueError`, while creating an immutable value prevents future
updates.

In [17]:
immutable = Value(data=[1, 2, 3], shape=Shape.vector(3), dtype=DType.INT32, mutable=False)

try:
    immutable.set([4, 5, 6])
except RuntimeError as e:
    print(f"Error: {e}")
# end try


Error: Trying to modify an immutable Value.
